In [1]:
import torch

In [2]:
tn = 5

In [5]:
self_attn = torch.ones((tn, tn))
self_attn

tensor([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [6]:
self_attn.tril()

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])

In [12]:
other_attn = torch.zeros((2*tn,2*tn))
print(other_attn.numpy())

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [ ]:
"""

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 0. 0.]
 [0. 1. 0. 1. 0. 0. 1. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 0.]
 [0. 1. 0. 1. 0. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0. 1. 0. 1. 0. 1.]]

"""

In [4]:
import torch

# Initialize a 10x10 tensor with zeros
matrix = torch.zeros(8, 8)

# Loop over each row index
for i in range(8):
    if i >= 1:  # Skip row 0, which remains all zeros
        if i % 2 == 1:  # Odd rows
            # Set ones at odd indices from 1 to i (inclusive)
            # range(1, i+1, 2) gives 1, 3, 5, ..., up to <= i
            matrix[i, 1:i+1:2] = 1
        else:  # Even rows (i >= 2)
            # Set ones at odd indices from 1 to i-1 (exclusive)
            # For i=2, this is empty; for i=4, it’s [1]; etc.
            if i-1 > 1:
                matrix[i, 1:i-1:2] = 1
            # Set ones at j = i and j = i+1, ensuring j <= 9
            matrix[i, i] = 1
            if i + 1 <= 7:
                matrix[i, i + 1] = 1

# Convert to float for consistency with the input format (optional)
matrix = matrix.float()

# Print to verify (optional)
print(matrix)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 1., 0., 0.],
        [0., 1., 0., 1., 0., 1., 0., 0.],
        [0., 1., 0., 1., 0., 0., 1., 1.],
        [0., 1., 0., 1., 0., 1., 0., 1.]])


In [21]:
import torch

def create_attention_mask_matrix(tn):
    # Initialize the 10x10 matrix
    matrix = torch.zeros(tn, tn)

    # Define odd columns mask (j=1,3,5,7,9)
    odd_cols = torch.arange(tn) % 2 == 1  # [False, True, False, True, ..., True]

    # Define row indices
    odd_rows = torch.tensor([x for x in range(1, tn) if x%2==1])
    even_rows = torch.tensor([x for x in range(1, tn) if x%2==0])

    # For odd rows: ones at odd columns j ≤ i
    # Use tril to get 1s where j ≤ i, then mask with odd columns
    tril_matrix = torch.tril(torch.ones(tn, tn))
    matrix[odd_rows, :] = tril_matrix[odd_rows, :] * odd_cols

    # For even rows: ones at odd j ≤ i-2, plus j=i and j=i+1
    # Use tril with diagonal=-2 for j ≤ i-2, mask with odd columns
    tril_minus2 = torch.tril(torch.ones(tn, tn), diagonal=-2)
    matrix[even_rows, :] = tril_minus2[even_rows, :] * odd_cols
    # Set specific positions for even rows
    matrix[even_rows, even_rows] = 1    # j = i
    matrix[even_rows, even_rows + 1] = 1  # j = i+1

    # Ensure float type (optional)
    # matrix = matrix
    return matrix[1:, 1:].bool()

In [23]:
create_attention_mask_matrix(8)

tensor([[ True, False, False, False, False, False, False],
        [False,  True,  True, False, False, False, False],
        [ True, False,  True, False, False, False, False],
        [ True, False, False,  True,  True, False, False],
        [ True, False,  True, False,  True, False, False],
        [ True, False,  True, False, False,  True,  True],
        [ True, False,  True, False,  True, False,  True]])

In [26]:
tril_minus2[even_rows, :]

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.]])

In [27]:
odd_cols

tensor([False,  True, False,  True, False,  True, False,  True])

In [28]:
tril_minus2[even_rows, :] * odd_cols

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 1., 0., 0., 0., 0.]])